TextRank: https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/

In [1]:
from parameters import Parameters
from transformers import BertTokenizer, BertForPreTraining
import random
from torch.utils.data import DataLoader, RandomSampler
import torch
import numpy as np, pandas as pd

In [3]:
0 + torch.FloatTensor([1,2])

tensor([1., 2.])

In [2]:
z = torch.FloatTensor([1,2,3]).to('cuda')

In [49]:
df.desc_clean_msg_stripped_locs[df.desc_clean=="""This well-presented 3BR home is in a terrific location in highly-desirable Dingley Village and needs nothing more to do than move-in and relax. Surrounded by lush sub-tropical plants and high fences, timber decks, paved walkways and low maintenance gardens. The front master bedroom has French doors opening out to a private front garden. There is a walk-around-robe and large, sparkling ensuite that has been recently renovated. The L-shaped front lounge opens out to a private deck plus there is a truly massive family room that extends from the kitchen to the back of the house. Both large living areas have beautifully polished floors. At the rear is a covered alfresco deck which is perfect for entertaining family and friends. The kitchen is bright and fresh with a feature window-splashback and stone features plus stainless appliances including dishwasher. There are two ample bedrooms each with BIR plus a renovated family bathroom, a separate laundry and large carport. This clean and tidy home has ducted gas heating, split system AC, lots of space and little maintenance. Walk to local shops, primary school and parks or take an short trip to DFO Kingston Plaza, Southland and the beach.."""][0]

"[{'entity_group': 'MISC', 'score': 0.9864088, 'word': 'French', 'start': 143, 'end': 149}, {'entity_group': 'LOC', 'score': 0.79081476, 'word': 'DFO Kingston Plaza', 'start': 1011, 'end': 1029}, {'entity_group': 'LOC', 'score': 0.99347866, 'word': 'Southland', 'start': 1031, 'end': 1040}]"

In [59]:
def find_potential_masks(text, locs, loc_score_thresh):
        hot_masks = []
        loc_masks = []
        # for sentence in sent_tokenize(text):
        #     for word in word_tokenize(sentence):
        #         if word.lower() in self.hotwords.keys():
        #             hot_masks.append({'word':word, 
        #                               'prob':self.hotwords[word.lower()], 
        #                               'tokens':self.hotword_tokens[word.lower()]})
        # entities = self.ner(text)
        for i,entity in enumerate(locs):
            if entity['entity_group'] == 'LOC' and entity['score'] > loc_score_thresh and not '#' in entity['word']:
                    loc_masks.append({'word':entity['word'], 
                                      'prob':entity['score'], 
                                      'tokens':tokenizer.encode(entity['word'])[1:-1]})

        return hot_masks, loc_masks

In [124]:
import ast
descs = []
locs = []
for i,(desc,loc) in enumerate(zip(df.desc_clean, df.desc_clean_msg_stripped_locs)):
    if isinstance(desc,str) and isinstance(loc,str) and isinstance(ast.literal_eval(loc), list):
        descs.append(desc)
        locs.append(ast.literal_eval(loc))

In [143]:
def collate_func(batch):
    descs, locs = batch
    return list(zip(descs, locs))

In [144]:
dataloader = DataLoader(dataset=list(zip(descs, locs)), batch_size=2, drop_last=True, collate_fn=collate_func)

In [152]:
for batch in dataloader:
    descs, locs = batch
    for i,(desc,loc) in enumerate(zip(descs, locs)):
        x,y = find_potential_masks(desc, loc, 0.75)
        break
    break

In [154]:
y

[{'word': 'DFO Kingston Plaza',
  'prob': 0.79081476,
  'tokens': [1040, 14876, 9803, 8232]},
 {'word': 'Southland', 'prob': 0.99347866, 'tokens': [29121]}]

In [128]:
batch[1]

[{'entity_group': ['MISC', 'LOC'],
  'score': tensor([0.9864, 0.9911], dtype=torch.float64),
  'word': ['French', 'Northland'],
  'start': tensor([143,  27]),
  'end': tensor([149,  36])},
 {'entity_group': ['LOC', 'ORG'],
  'score': tensor([0.7908, 0.6385], dtype=torch.float64),
  'word': ['DFO Kingston Plaza', 'Bir'],
  'start': tensor([1011,  667]),
  'end': tensor([1029,  670])},
 {'entity_group': ['LOC', 'LOC'],
  'score': tensor([0.9935, 0.6741], dtype=torch.float64),
  'word': ['Southland', 'Darebin'],
  'start': tensor([1031, 1101]),
  'end': tensor([1040, 1108])}]

In [3]:
x=torch.Tensor([1,2,3])
y=torch.Tensor([4,5,6])
x,y += torch.Tensor([1,1,1])

SyntaxError: 'tuple' is an illegal expression for augmented assignment (Temp/ipykernel_29528/631169534.py, line 3)

In [55]:
batch_size = 16
train_test_split = 0.8
params = Parameters()
max_input_length = params.max_input_length
model_name = "bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForPreTraining.from_pretrained(model_name)

Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
df = pd.read_csv(params.dataset_filename, low_memory=False)
descs = [x for x in df.desc_clean.tolist() if isinstance(x,str)]

N = len(descs)
N_train = int(N * train_test_split)
N_val = N - N_train
d_train, d_val = torch.utils.data.dataset.random_split(descs, [N_train, N_val])
dl_train = DataLoader(dataset=d_train, batch_size=batch_size, sampler=RandomSampler(d_train), drop_last=True, collate_fn=collate_func)
dl_val = DataLoader(dataset=d_val, batch_size=batch_size, sampler=RandomSampler(d_val), drop_last=True, collate_fn=collate_func)

NameError: name 'train_test_split' is not defined

In [16]:
import ast
descs = df.desc_clean
locs = df.mask_locs
dataset = []
s = 0
for pair in zip(descs,locs):
    desc,loc = pair
    if isinstance(desc,str):
        if isinstance(loc, str):
            if isinstance(ast.literal_eval(loc), list):
                s += 1
        
# dataset = [(desc,ast.literal_eval(loc)) for desc,loc in zip(descs, locs) if isinstance(desc,str)]

In [17]:
s

49226

In [13]:
def random_word(tokens, tokenizer):
    output_label = []

    for i, token in enumerate(tokens):
        prob = random.random()
        # mask token with 15% probability
        if prob < 0.15:
            prob /= 0.15

            # 80% randomly change token to mask token
            if prob < 0.8:
                tokens[i] = tokenizer.mask_token

            # 10% randomly change token to random token
            elif prob < 0.9:
                tokens[i] = random.choice(list(tokenizer.vocab.items()))[0]
            # -> rest 10% randomly keep current token
            output_label.append(tokenizer.vocab[token])
        else:
            # no masking token (will be ignored by loss function later)
            output_label.append(-1)

    return tokens, output_label

def _truncate_seq_pair(tokens_a, tokens_b, max_length=max_input_length):
    """Truncates a sequence pair in place to the maximum length."""
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

def convert_example_to_features(tokens_a, tokens_b, max_seq_length, tokenizer):
    _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)

    tokens_a, t1_label = random_word(tokens_a, tokenizer)
    tokens_b, t2_label = random_word(tokens_b, tokenizer)
    # concatenate lm labels and account for CLS, SEP, SEP

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0   0   0   0  0     0 0
    #

    tokens =      [tokenizer.cls_token] + tokens_a +             [tokenizer.sep_token] + tokens_b +              [tokenizer.sep_token]
    segment_ids = [0] +      (len(tokens_a) * [0]) + [0] +       (len(tokens_b) * [1]) + [1] 
    lm_label_ids = [-1] + t1_label + [-1] + t2_label + [-1]

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    pad_amount = max_seq_length - len(input_ids)
    input_mask = [1] * len(input_ids) + [0] * pad_amount
    input_ids += [0] * pad_amount
    segment_ids += [0] * pad_amount
    lm_label_ids += [-1] * pad_amount

    return input_ids, input_mask, segment_ids, lm_label_ids

def collate_func(bodies):
    bodies_tokenized = [tokenizer.tokenize(body) for body in bodies]

    max_length = max_input_length
    half_length = int(max_length/2)

    is_next_labels = []
    mid_point = int(len(bodies)/2)
    batch_ids, batch_mask, batch_segments, batch_lm_label_ids, batch_is_next = [], [], [], [], []
    for i in range(mid_point):
        is_next = 1 if random.random() < 0.5 else 0

        tokens_a = bodies_tokenized[i]
        if is_next == 0:
            tokens_b = bodies_tokenized[i]
        else:
            tokens_b = bodies_tokenized[i+mid_point]
        half_length_a = min(half_length, int(len(tokens_a) / 2))
        half_length_b = min(half_length, int(len(tokens_b) / 2))
        max_length_b = min(max_length, int(len(tokens_b)))
        tokens_a = tokens_a[:half_length_a]
        tokens_b = tokens_b[half_length_b:max_length_b]
        input_ids, input_mask, segment_ids, lm_label_ids = convert_example_to_features(tokens_a, tokens_b, max_length, tokenizer)

        batch_ids.append(input_ids)
        batch_mask.append(input_mask)
        batch_segments.append(segment_ids)
        batch_lm_label_ids.append(lm_label_ids)
        batch_is_next.append(is_next)

    batch_ids = torch.LongTensor(batch_ids)
    batch_mask = torch.LongTensor(batch_mask)
    batch_segments = torch.LongTensor(batch_segments)
    batch_lm_label_ids = torch.LongTensor(batch_lm_label_ids)
    batch_is_next = torch.LongTensor(batch_is_next)

    return batch_ids, batch_mask, batch_segments, batch_lm_label_ids, batch_is_next

In [82]:
for ib, batch in enumerate(dl_train):
    batch = tuple(t for t in batch)
    input_ids, input_mask, segment_ids, lm_label_ids, is_next = batch
    out = model(input_ids=input_ids, attention_mask=input_mask, token_type_ids=segment_ids)#, labels=lm_label_ids, next_sentence_label=is_next, return_dict=True)
    mlm_logits = out.prediction_logits
    is_next_logits = out.seq_relationship_logits
    is_next_acc = is_next.eq(torch.argmax(is_next_logits, dim=1)).float().sum().item()
    num_predicts_mlm = (~lm_label_ids.eq(-1)).sum().item()
    mlm_acc = (lm_label_ids.view(-1).eq(torch.argmax(mlm_logits,dim=2).view(-1)).float().sum()).item()
    break

In [87]:
(len(dl_train)*16)*0.03*16

19353.6

In [1]:
512*0.15*16

1228.8

In [2]:
(~lm_label_ids.eq(-1)).sum()

NameError: name 'lm_label_ids' is not defined

In [74]:
lm_label_ids

tensor([[  -1,   -1,   -1,  ...,   -1,   -1,   -1],
        [  -1,   -1,   -1,  ...,   -1,   -1,   -1],
        [  -1,   -1,   -1,  ...,   -1,   -1,   -1],
        ...,
        [  -1, 1037,   -1,  ...,   -1,   -1,   -1],
        [  -1,   -1, 2307,  ...,   -1,   -1,   -1],
        [  -1,   -1,   -1,  ...,   -1,   -1,   -1]])

In [75]:
is_next

tensor([0, 0, 0, 0, 1, 0, 0, 1])

In [76]:
is_next_acc = is_next.eq(torch.argmax(is_next_logits, dim=1)).float().mean().item()

In [78]:
num_predicts = (~lm_label_ids.eq(-1)).sum().item()
mlm_acc = (lm_label_ids.view(-1).eq(torch.argmax(mlm_logits,dim=2).view(-1)).float().sum()/num_predicts).item()

In [79]:
is_next_acc

0.75

In [80]:
mlm_acc

0.5214285850524902